# Collaborative Filtering with [Surprise](https://surpriselib.com/) Toolkit

In [1]:
import pandas as pd
import numpy as np
import random
from surprise import Dataset, Reader, accuracy, dump
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import SVD, SVDpp, BaselineOnly
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
import surprise_helper

## Preparation

In [2]:
rates = pd.read_csv("rates.csv")
rates.head(3)

,user_id,display_name,course_id,rate
0,14177,Jacynthe,1055720,5.0
1,24654,Norval,1055720,5.0
2,14484,Jany,1055720,4.0


In [3]:
courses = pd.read_csv("courses.csv")
courses.head(3)

,id,title,category,course_url
0,9287,Microsoft Excel 2010 Course Beginners/ Interme...,Office Productivity,/course/excel-tutorial/
1,9385,Microsoft Excel 2010: Advanced Training,Office Productivity,/course/advanced-excel/
2,9711,Beginner PHP and MySQL Tutorial,Development,/course/php-mysql-tutorial/


In [4]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(rates[["user_id", "course_id", "rate"]], reader)

## Model Selection

### Basic Collaborative Filtering Algorithms
#### BaselineOnly

$baseline = b_{ui} = \text{average rating of all courses + bias of user u + bias of item i} $

$b_u$ and $b_i$ minimized by using objective function.

In [5]:
_ = cross_validate(BaselineOnly(verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm BaselineOnly on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.6919  0.6939  0.6884  0.6914  0.0023  
MAE (testset)     0.4729  0.4753  0.4727  0.4736  0.0012  
Fit time          0.51    0.58    0.61    0.57    0.04    
Test time         0.28    0.38    0.35    0.34    0.04    


### Memory Based Collaborative Filtering

#### Basic KNN

Simple k-NN method by using similarity metric.

User based: Example -> users similar to you bought Y

$\large est_{ui} = \frac{\text{sum_nearest_k_v(similarity between user u and v * rate given to item i by user v)}}{\text{sum_nearest_k_v(similarity between user u and v)}}$

Item based: Example ->  users who bought X, also bought Y

$\large est_{ui} = \frac{\text{sum_nearest_k_j(similarity between item i and j * rate given to item j by user u)}}{\text{sum_nearest_k_j(similarity between item i and j)}}$

In [6]:
_ = cross_validate(KNNBasic(sim_options={"name": "msd", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNBasic(sim_options={"name": "cosine", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNBasic(sim_options={"name": "pearson", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNBasic(sim_options={"name": "pearson_baseline", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7971  0.8081  0.7949  0.8000  0.0058  
MAE (testset)     0.4737  0.4786  0.4713  0.4745  0.0030  
Fit time          0.15    0.20    0.20    0.19    0.02    
Test time         0.70    0.73    0.69    0.71    0.02    
Evaluating RMSE, MAE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7994  0.7972  0.7928  0.7965  0.0028  
MAE (testset)     0.4748  0.4745  0.4735  0.4743  0.0006  
Fit time          0.26    0.30    0.31    0.29    0.02    
Test time         0.63    0.72    0.70    0.68    0.04    
Evaluating RMSE, MAE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7519  0.7516  0.7500  0.7512  0.0008  
MAE (testset)     0.4867  0.4874  0.4869  0.4870  0.0003  
Fit time          0.34    0.42    0.39    0.38    0.03  

#### KNN with Means

A k-NN method similar to previous one but takes into account the mean ratings of each user.

User based:

$\large est_{ui} = \text{mean rating of user u} + \frac{\text{sum_nearest_k_v(similarity between user u and v * (rate given to item i by user v - mean rating of user v)}}{\text{sum_nearest_k_v(similarity between user u and v)}}$

Item based:

$\large est_{ui} = \text{mean rating of item i} + \frac{\text{sum_nearest_k_j(similarity between item i and j * (rate given to item j by user u - mean rating of item j))}}{\text{sum_nearest_k_j(similarity between item i and j)}}$

In [7]:
_ = cross_validate(KNNWithMeans(sim_options={"name": "msd", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNWithMeans(sim_options={"name": "cosine", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNWithMeans(sim_options={"name": "pearson", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNWithMeans(sim_options={"name": "pearson_baseline", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm KNNWithMeans on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7830  0.7834  0.7812  0.7825  0.0010  
MAE (testset)     0.4827  0.4835  0.4819  0.4827  0.0007  
Fit time          0.20    0.25    0.26    0.23    0.03    
Test time         0.74    0.74    0.69    0.72    0.03    
Evaluating RMSE, MAE of algorithm KNNWithMeans on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7787  0.7795  0.7812  0.7798  0.0010  
MAE (testset)     0.4815  0.4821  0.4820  0.4818  0.0003  
Fit time          0.31    0.35    0.37    0.34    0.03    
Test time         0.75    0.76    0.76    0.76    0.00    
Evaluating RMSE, MAE of algorithm KNNWithMeans on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7391  0.7392  0.7439  0.7407  0.0022  
MAE (testset)     0.4811  0.4806  0.4826  0.4814  0.0009  
Fit time          0.38    0.43    0.44    0.

#### KNN with Z Score

A k-NN method similar to basic k-NN one but takes into account the z-score normalization of each user.

User based:

$\large est_{ui} = \text{mean rating of user u} + \text{std of user u} * \frac{\text{sum_nearest_k_v(similarity between user u and v * (rate given to item i by user v - mean rating of user v) / std of user v)}}{\text{sum_nearest_k_v(similarity between user u and v)}}$

Item based:

$\large est_{ui} = \text{mean rating of item i} + \text{std of item i} * \frac{\text{sum_nearest_k_j(similarity between item i and j * (rate given to item j by user u - mean rating of item j) / std of item j)}}{\text{sum_nearest_k_j(similarity between item i and j)}}$

In [8]:
_ = cross_validate(KNNWithZScore(sim_options={"name": "msd", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNWithZScore(sim_options={"name": "cosine", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNWithZScore(sim_options={"name": "pearson", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNWithZScore(sim_options={"name": "pearson_baseline", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm KNNWithZScore on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7924  0.7965  0.7914  0.7935  0.0022  
MAE (testset)     0.4795  0.4809  0.4772  0.4792  0.0015  
Fit time          0.33    0.37    0.38    0.36    0.02    
Test time         0.79    0.77    0.79    0.79    0.01    
Evaluating RMSE, MAE of algorithm KNNWithZScore on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7931  0.7858  0.7946  0.7911  0.0038  
MAE (testset)     0.4814  0.4780  0.4800  0.4798  0.0014  
Fit time          0.45    0.48    0.50    0.48    0.02    
Test time         0.78    0.77    0.72    0.76    0.03    
Evaluating RMSE, MAE of algorithm KNNWithZScore on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7437  0.7424  0.7413  0.7425  0.0010  
MAE (testset)     0.4806  0.4790  0.4807  0.4801  0.0008  
Fit time          0.51    0.59    0.56   

#### KNN with Baseline

A k-NN method similar to basic k-NN one but takes into account a *baseline* rating.

$baseline = b_{ui} = \text{average rating of all courses + bias of user u + bias of item i} $

User based:

$\large est_{ui} = \text{baseline rating of user u to item i} + \frac{\text{sum_nearest_k_v(similarity between user u and v * (rate given to item i by user v - baseline rating of user v to item i))}}{\text{sum_nearest_k_v(similarity between user u and v)}}$

Item based:

$\large est_{ui} = \text{baseline rating of user u to item i} + \frac{\text{sum_nearest_k_j(similarity between item i and j * (rate given to item j by user u - baseline rating of user u to item j))}}{\text{sum_nearest_k_j(similarity between item i and j)}}$

In [9]:
_ = cross_validate(KNNBaseline(sim_options={"name": "msd", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNBaseline(sim_options={"name": "cosine", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNBaseline(sim_options={"name": "pearson", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNBaseline(sim_options={"name": "pearson_baseline", "user_based": False}, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7764  0.7775  0.7845  0.7795  0.0036  
MAE (testset)     0.4774  0.4778  0.4804  0.4785  0.0014  
Fit time          0.70    0.72    0.74    0.72    0.01    
Test time         0.80    0.81    0.80    0.80    0.01    
Evaluating RMSE, MAE of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7772  0.7768  0.7743  0.7761  0.0013  
MAE (testset)     0.4789  0.4781  0.4758  0.4776  0.0013  
Fit time          0.79    0.84    0.83    0.82    0.02    
Test time         0.80    0.79    0.73    0.78    0.03    
Evaluating RMSE, MAE of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7399  0.7312  0.7319  0.7344  0.0039  
MAE (testset)     0.4768  0.4733  0.4726  0.4742  0.0018  
Fit time          0.86    0.93    0.94    0.91 

Best results for all these 4 models are achieved with pearson similarity metric. Overall best result is achieved with KNNBaseline.

### Model-based Collaborative Filtering (Matrix Factorization)

#### SVD

$\text{overall interest} = o_i = q_i^T\cdot p_u, q_i = \text{vector for item i}, p_u = \text{vector for user u}$

$est_{ui} = \text{mean of all courses + baseline of item + baseline of user + overall interest}$

Parameters learned by minimizing objective function.

In [10]:
_ = cross_validate(SVD(verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.6949  0.7002  0.7005  0.6985  0.0026  
MAE (testset)     0.4722  0.4717  0.4738  0.4726  0.0009  
Fit time          1.78    1.77    1.83    1.80    0.02    
Test time         0.47    0.53    0.54    0.51    0.03    


#### SVD++

An extension of SVD taking into account implicit ratings. For this implementation, an implicit rating describes the fact that a user u rated an item j, regardless of the rating value.

In [11]:
_ = cross_validate(SVDpp(verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.6999  0.6950  0.6938  0.6962  0.0027  
MAE (testset)     0.4698  0.4676  0.4673  0.4682  0.0011  
Fit time          2.00    2.03    2.04    2.02    0.01    
Test time         1.25    1.24    1.26    1.25    0.01    


These matrix factorization models outperformed the memory based methods.

We tested 7 different collaborative filtering methods. We choose 3 of them to fine-tune and see which one is the best. The selected ones are BaselineOnly, KNNBaseline and SVD. We selected SVD because it is slightly faster than SVD++.

### Parameter Search

In [12]:
param_grid_base = {"bsl_options": {
    "method": ["sgd"], 
    "reg":[0.01, 0.02, 0.03], 
    "learning_rate": [0.005, 0.01, 0.02], 
    "n_epochs": [10, 20, 30]},
    "verbose": [False]
}
gs_base = GridSearchCV(BaselineOnly, param_grid_base, measures=["rmse", "mae"], cv=5, joblib_verbose=0)
gs_base.fit(data)
print("BaselineOnly SGD Grid Search")
print(gs_base.best_score["rmse"])
print(gs_base.best_params["rmse"])

BaselineOnly SGD Grid Search
0.6885029389204965
{'bsl_options': {'method': 'sgd', 'reg': 0.03, 'learning_rate': 0.005, 'n_epochs': 20}, 'verbose': False}


Best hyperparameter values for baseline are *{'method': 'sgd', 'reg': 0.03, 'learning_rate': 0.005, 'n_epochs': 20}*.

In [13]:
"""
param_grid_knn = {
    "k": [20,30,40], 
    "sim_options ": {"name": "pearson", "user_based": False},
    "verbose": [False]
}
gs_knn = GridSearchCV(KNNBaseline, param_grid_knn, measures=["rmse", "mae"], cv=5)
gs_knn.fit(data)
print("KNNWithBaseline Grid Search")
print(gs_knn.best_score["rmse"])
print(gs_knn.best_params["rmse"])
"""

# Parameter tuning for KNNBaseline could not done due to the memory limitations. 
# Instead of that we tried cross validation with baseline hypermeters to see if they have positive effect on model.

bsl_opt = {'method': 'sgd', 'reg': 0.03, 'learning_rate': 0.005, 'n_epochs': 20}
sim_opt = {"name": "pearson", "user_based": False}

_ = cross_validate(KNNBaseline(k=20, bsl_options=bsl_opt, sim_options=sim_opt, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNBaseline(k=30, bsl_options=bsl_opt, sim_options=sim_opt, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
_ = cross_validate(KNNBaseline(k=40, bsl_options=bsl_opt, sim_options=sim_opt, verbose=False), data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7307  0.7310  0.7358  0.7325  0.0024  
MAE (testset)     0.4707  0.4700  0.4728  0.4712  0.0012  
Fit time          1.20    1.24    1.23    1.22    0.02    
Test time         0.64    0.63    0.64    0.64    0.00    
Evaluating RMSE, MAE of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7306  0.7325  0.7388  0.7339  0.0035  
MAE (testset)     0.4694  0.4717  0.4751  0.4721  0.0023  
Fit time          1.19    1.24    1.25    1.23    0.03    
Test time         0.69    0.68    0.77    0.72    0.04    
Evaluating RMSE, MAE of algorithm KNNBaseline on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7299  0.7352  0.7357  0.7336  0.0026  
MAE (testset)     0.4693  0.4722  0.4730  0.4715  0.0016  
Fit time          1.20    1.22    1.24    1.22 

From the results for KNNBaseline we can see that changing the k value and newly set baseline options did not affect the accuracy values.

In [14]:
param_grid_svd = {"n_factors": [100, 200],
                  "n_epochs": [10, 20], 
                  "lr_all": [0.005, 0.01, 0.02],
                  "reg_all": [0.01, 0.02, 0.03],
}
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=["rmse", "mae"], cv=5)
gs_svd.fit(data)
print("SVD Grid Search")
print(gs_svd.best_score["rmse"])
print(gs_svd.best_params["rmse"])

SVD Grid Search
0.6945597792676461
{'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.03}


Best hyperparameter values for SVD are *{'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.03}*.

***

Lets test these models with train-test sets. 

In [15]:
trainset, testset = train_test_split(data, random_state=78, test_size=0.2)

In [16]:
pred = BaselineOnly(bsl_options:={'method': 'sgd', 'reg': 0.03, 'learning_rate': 0.005, 'n_epochs': 20}, verbose=False).fit(trainset).test(testset)
print("RMSE values of BaselineOnly:", accuracy.rmse(pred))

RMSE: 0.6949
RMSE values of BaselineOnly: 0.6948999191927382


In [17]:
pred = KNNBaseline(bsl_options=bsl_opt, sim_options=sim_opt, verbose=False).fit(trainset).test(testset)
print("RMSE values of KNNBaseline:", accuracy.rmse(pred))

RMSE: 0.7538
RMSE values of KNNBaseline: 0.7538073470017157


In [18]:
pred = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.03).fit(trainset).test(testset)
print("RMSE values of SVD:", accuracy.rmse(pred))

RMSE: 0.7006
RMSE values of SVD: 0.7006290645236952


## Recommendation

We decided check SVD and BaselineOnly to see how well they recommend courses to users.

In [5]:
full_trainset = data.build_full_trainset()

In [6]:
bsl = BaselineOnly(bsl_options={'method': 'sgd', 'reg': 0.03, 'learning_rate': 0.005, 'n_epochs': 20}, verbose=False)
svd = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.03)
bsl.fit(full_trainset)
svd.fit(full_trainset)

In [7]:
# causes memory errors, instead calculate anti testset for the user
#anti_testset = full_trainset.build_anti_testset()
anti_testset_user, user_id = surprise_helper.get_anti_testset_user(full_trainset)
print("User ID:", user_id)
print("Display Name:", rates[rates["user_id"] == user_id]["display_name"].unique()[0])

User ID: 30179
Display Name: Sathyababu


In [8]:
user_pred_svd = svd.test(anti_testset_user)
user_pred_bsl = bsl.test(anti_testset_user)

In [9]:
print("Courses Taken By The User")
surprise_helper.get_user_courses(user_id, rates, courses)

Courses Taken By The User


,id,title,category,course_url
10,13391,The Neuroscience of Reframing & How to Do It,Teaching & Academics,/course/the-neuroscience-of-reframing-and-how-...
233,277654,Master your brain: Neuroscience for personal d...,Personal Development,/course/my-brain-and-i/
415,466736,Stress Management: 40+ easy ways to deal with ...,Personal Development,/course/stress-management-40-easy-ways-to-deal...
772,789146,Professional Life Coach Certification & Guide ...,Personal Development,/course/life-coaching-a-complete-guide/
1185,1091520,Cognitive Behavioural Therapy (CBT) Practition...,Health & Fitness,/course/cognitive-behavioural-therapy-online-c...
1650,1415934,Management Skills - Team Leadership Skills Mas...,Business,/course/leadership-and-lean-management-masterc...
2250,2323470,Business Communication Skills: Business Writin...,Personal Development,/course/professional-communication-and-busines...


In [10]:
print("Recommended Courses by SVD")
surprise_helper.get_top_n(user_pred_svd, courses)

Recommended Courses by SVD


,id,title,category,course_url
323,379080,Programador web: Cookies y Sesiones en PHP,Development,/course/programador-web-php-cookies-y-sesiones/
524,567366,Launch Your Startup Entrepreneur Business,Business,/course/entrepreneurs_expert/
1456,1292182,Fully Accredited Reflexology Diploma - Heal Vi...,Personal Development,/course/fully-accredited-reflexology-course-he...
2106,2024924,"Reiki nível 2 Usui, Kahuna, Tibetano",Lifestyle,/course/reiki-nivel-2-usui-kahuna-tibetano/
2433,2846906,Product Owner Certification Mock Exams 2022 - ...,IT & Software,/course/product-owner-certificate-training-200...


In [11]:
print("Recommended Courses by BaselineOnly")
surprise_helper.get_top_n(user_pred_bsl, courses)

Recommended Courses by BaselineOnly


,id,title,category,course_url
479,523406,Como criar um curso online na Udemy - Unofficial,Teaching & Academics,/course/criarcurso/
524,567366,Launch Your Startup Entrepreneur Business,Business,/course/entrepreneurs_expert/
997,949316,Foundations for Mastering Watercolor Painting ...,Lifestyle,/course/foundations-for-mastering-watercolor-p...
2540,4119364,Vue.js: De cero a experto,Development,/course/vuejs-fh/
2548,4599580,Principios SOLID y Clean Code,Development,/course/solid-clean/


Sometimes when we look at the recommended courses it feels wrong. Since these methods only takes ratings into consideration, recommended courses are the best according to only accuracy values. Recommendations can be done better with using other informations such as user informations and implicit ratings and it is called *Content-based Filtering*. 

### Similar Courses with KNN

We can use k-NN based methods to see the similar courses & users.

In [12]:
knn = KNNBaseline(sim_options={"name": "pearson", "user_based": False}, verbose=False)
knn.fit(full_trainset)

In [13]:
similar_courses_raw_ids, course_id = surprise_helper.get_nearest_neighbors(full_trainset, knn)

In [14]:
print("Course")
courses[courses["id"]==course_id]

Course


,id,title,category,course_url
2016,1844356,Complete guide to ASP.NET Core MVC (.NET 6),Development,/course/complete-aspnet-core-21-course/


In [15]:
print("Similar Courses")
courses[courses["id"].isin(similar_courses_raw_ids)]

Similar Courses


,id,title,category,course_url
81,65330,Web Development By Doing: HTML / CSS From Scratch,Development,/course/web-development-learn-by-doing-html5-c...
792,802574,AZ-204 Developing Solutions for Microsoft Azur...,IT & Software,/course/70532-azure/
1726,1480202,RESTful Web API - The Complete Guide (.NET7 API),Development,/course/restful-api-with-asp-dot-net-core-web-...
1773,1527300,Learn Python & Ethical Hacking From Scratch,Development,/course/learn-python-and-ethical-hacking-from-...
2287,2395488,React Front To Back,Development,/course/modern-react-front-to-back/


## Recommendation with Using Ratings Predicted From Comments

In this part, we tried to recommend courses by using the predicted ratings that we prepared with comment analysis. We assigned a rating to an entry based on the comment's positivity and negativity status. 

In [16]:
rates_pr = pd.read_csv("rates_pr.csv")
rates_pr.head(3)

,user_id,display_name,course_id,rate
0,27767,Venkateswara rao,1458490,3.9
1,8336,Felipe Rodriguez,1248866,3.6
2,6232,Derli,1623098,1.4


In [17]:
courses_pr = pd.read_csv("courses_pr.csv")
courses_pr.head(3)

,id,title,category,language,instructor_name
0,9287,Microsoft Excel 2010 Course Beginners/ Interme...,Office Productivity,English,Infinite Skills
1,9711,Beginner PHP and MySQL Tutorial,Development,English,Infinite Skills
2,10449,"Spanish 1-4: Beginner, Elementary, Intermediat...",Teaching & Academics,English,Simple3 L


In [18]:
print(rates_pr.shape, courses_pr.shape)

(6807, 4) (1763, 5)


In [19]:
reader_pr = Reader(rating_scale=(0, 5))
data_pr = Dataset.load_from_df(rates_pr[["user_id", "course_id", "rate"]], reader_pr)

In [20]:
svd_pr = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.03)
_ = cross_validate(svd_pr, data_pr, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9286  0.9594  0.9747  0.9542  0.0192  
MAE (testset)     0.7658  0.7999  0.7997  0.7884  0.0160  
Fit time          0.06    0.05    0.05    0.05    0.00    
Test time         0.01    0.01    0.01    0.01    0.00    


In [21]:
full_trainset_pr = data_pr.build_full_trainset()

In [22]:
anti_testset_user_pr, user_id_pr = surprise_helper.get_anti_testset_user(full_trainset_pr)
print("User ID:", user_id_pr)
print("Display Name:", rates_pr[rates_pr["user_id"] == user_id_pr]["display_name"].unique()[0])

User ID: 17101
Display Name: Massamar mocktar


In [23]:
svd_pr.fit(full_trainset_pr)

In [24]:
user_pred_pr = svd_pr.test(anti_testset_user_pr)

In [25]:
print("Courses Taken By The User")
surprise_helper.get_user_courses(user_id_pr, rates_pr, courses_pr)

Courses Taken By The User


,id,title,category,language,instructor_name
968,1151632,Deep Learning A-Z™: Hands-On Artificial Neural...,Development,English,Kirill Eremenko


In [26]:
print("Recommended Courses by SVD")
surprise_helper.get_top_n(user_pred_pr, courses_pr)

Recommended Courses by SVD


,id,title,category,language,instructor_name
498,648826,The Complete Financial Analyst Course 2022,Finance & Accounting,English,365 Careers
348,473160,Web Design for Web Developers: Build Beautiful...,Development,English,Jonas Schmedtmann
1467,1906852,MongoDB - The Complete Developer's Guide 2022,Development,English,Academind by Maximilian Schwarzmüller
1473,1917546,Master the Coding Interview: Data Structures +...,Development,English,Andrei Neagoie
975,1163356,The Complete Project Management Fundamentals C...,Business,English,Mauricio Rubio - Agile Guru & Founder of Agile...
